In [52]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

In [3]:
data = pd.read_csv("data/airplane/airplane_price_dataset.csv")
data.head()

,Model,Üretim Yılı,Motor Sayısı,Motor Türü,Kapasite,Menzil (km),Yakıt Tüketimi (L/saat),Saatlik Bakım Maliyeti ($),Yaş,Satış Bölgesi,Fiyat ($)
0,Bombardier CRJ200,1987,2,Turbofan,50,3000,14.36,2185.43,36,Asya,1.285708e+07
1,Bombardier CRJ200,1997,2,Turbofan,50,3000,4.03,1202.08,26,Avrupa,1.391406e+07
2,Airbus A320,1988,2,Turbofan,180,6300,13.26,761.38,35,Avustralya,9.073570e+07
3,Boeing 737,2023,2,Turbofan,162,5700,14.61,592.63,0,Avustralya,1.366597e+08
4,Cessna 172,1985,1,Piston,4,1285,18.49,4245.99,38,Güney Amerika,2.037981e+05


Voici une description de chaque colonne du dataset ainsi qu'une proposition de renommage en français pour une meilleure lisibilité :  

### **Description des colonnes :**  
1. **Model** → *Modèle* : Le nom du modèle d'avion.  
2. **Üretim Yılı** → *Année de production* : L'année de fabrication de l'avion.  
3. **Motor Sayısı** → *Nombre de moteurs* : Le nombre total de moteurs équipant l'avion.  
4. **Motor Türü** → *Type de moteur* : Le type de moteur utilisé (ex. : Turbofan, Turboprop, etc.).  
5. **Kapasite** → *Capacité* : Le nombre maximal de passagers que l'avion peut transporter.  
6. **Menzil (km)** → *Autonomie (km)* : La distance maximale que l'avion peut parcourir sans ravitaillement en carburant (en kilomètres).  
7. **Yakıt Tüketimi (L/saat)** → *Consommation de carburant (L/h)* : La quantité de carburant consommée par heure de vol (en litres par heure).  
8. **Saatlik Bakım Maliyeti ($)** → *Coût de maintenance horaire ($)* : Le coût moyen d'entretien de l'avion par heure de vol (en dollars).  
9. **Yaş** → *Âge* : L'âge de l'avion en années.  
10. **Satış Bölgesi** → *Région de vente* : La région où l'avion est vendu (ex. : Asie, Europe, etc.).  
11. **Fiyat ($)** → *Prix ($)* : Le prix de vente de l'avion en dollars.  

In [54]:
# renommage des colonnes
data.rename(columns={
    "Model": "Modèle",
    "Üretim Yılı": "Année de production",
    "Motor Sayısı": "Nombre de moteurs",
    "Motor Türü": "Type de moteur",
    "Kapasite": "Capacité",
    "Menzil (km)": "Autonomie (km)",
    "Yakıt Tüketimi (L/saat)": "Consommation de carburant (L/h)",
    "Saatlik Bakım Maliyeti ($)": "Coût de maintenance horaire ($)",
    "Yaş": "Âge",
    "Satış Bölgesi": "Région de vente",
    "Fiyat ($)": "Prix ($)"
}, inplace=True)

# Vérification des noms de colonnes
print("Colonnes du DataFrame :", data.columns.tolist())

Colonnes du DataFrame : ['Modèle', 'Année de production', 'Nombre de moteurs', 'Type de moteur', 'Capacité', 'Autonomie (km)', 'Consommation de carburant (L/h)', 'Coût de maintenance horaire ($)', 'Âge', 'Région de vente', 'Prix ($)']


# 1. Compréhension et Préparation des Données
Avant tout, il est important d'explorer le dataset pour identifier d'éventuelles valeurs manquantes, outliers ou incohérences. Ici, nous allons supposer que le dataset est propre.
Nous devons aussi distinguer les variables numériques des variables catégorielles.

**Variables numériques** :

- Année de production
- Nombre de moteurs
- Capacité
- Autonomie (km)
- Consommation de carburant (L/h)
- Coût de maintenance horaire ($)
- Âge


**Variables catégorielles** :

- Modèle
- Type de moteur
- Région de vente

**La variable cible sera Prix ($).**

# 2. Séparation des Données
On divise le dataset en deux ensembles :

Ensemble d'entraînement : Pour entraîner et optimiser le modèle.
Ensemble de test : Pour évaluer la performance finale du modèle.


In [56]:
# Séparation des données
X = data.drop("Prix ($)", axis=1)
y = data["Prix ($)"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Prétraitement et Transformation
Afin de préparer les données pour l'apprentissage, on va :

Scaler les variables numériques avec un StandardScaler pour leur donner une échelle comparable.
Encoder les variables catégorielles avec un OneHotEncoder pour les transformer en variables numériques.
Pour automatiser ces transformations, on utilise un ColumnTransformer qui sera intégré dans un pipeline.

In [58]:
# Préparation du préprocesseur
numerical_features = [
    "Année de production",  
    "Nombre de moteurs",
    "Capacité",
    "Autonomie (km)",
    "Consommation de carburant (L/h)",
    "Coût de maintenance horaire ($)",
    "Âge"
]
categorical_features = ["Modèle", "Type de moteur", "Région de vente"]

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numerical_features),
        ("cat", OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

# 4. Choix du Modèle et Création d'un Pipeline
Pour un problème de régression comme celui-ci, XGBoost est un excellent choix grâce à sa robustesse et sa capacité à gérer les interactions complexes entre variables.
On intègre le préprocesseur et le modèle dans un pipeline, ce qui permet d'automatiser l'ensemble du flux de travail.

In [60]:
# Création du pipeline
xgb_model = XGBRegressor(objective="reg:squarederror", random_state=42)
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor", xgb_model)
])


# 5. Optimisation des Hyperparamètres
Pour améliorer les performances du modèle, il est essentiel d'optimiser certains hyperparamètres (par exemple, le nombre d'arbres, la profondeur maximale, le taux d'apprentissage, etc.).
On peut utiliser GridSearchCV pour explorer différentes combinaisons via une validation croisée.

In [63]:
# Recherche d'hyperparamètres
param_grid = {
    "regressor__n_estimators": [100, 200],
    "regressor__max_depth": [3, 5, 7],
    "regressor__learning_rate": [0.01, 0.1, 0.2]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring="neg_mean_squared_error", error_score='raise')
grid_search.fit(X_train, y_train)

print("Meilleurs paramètres :", grid_search.best_params_)

Meilleurs paramètres : {'regressor__learning_rate': 0.1, 'regressor__max_depth': 3, 'regressor__n_estimators': 100}


# 6. Évaluation du Modèle
Une fois le modèle optimisé, on l'évalue sur l'ensemble de test à l'aide de métriques telles que le RMSE (Root Mean Squared Error) et le R².

In [84]:
# Évaluation sur le jeu de test
y_pred = grid_search.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)

print("RMSE :", rmse)
print("R² :", r2)
print("MAE :", mae)
print("MAPE :", mape * 100, "%")

RMSE : 31091684.07172641
R² : 0.9819152421253083
MAE : 16664613.832613593
MAPE : 71.79805584148 %


In [75]:
data["Prix ($)"].max()

978213228.63

In [77]:
data["Prix ($)"].max()

145814.79

In [79]:
data["Prix ($)"].mean()

198833649.57071987

# Interprétation des Résultats

Ces résultats suggèrent que, malgré un **RMSE** de 31 millions, le modèle est très performant par rapport à l'ampleur des valeurs (avec une erreur relative d'environ 15 % et un **R²** très élevé).

Les métriques **MAE** et **MAPE** donnent deux points de vue différents sur l'erreur de votre modèle :

- **MAE ≈ 16,66 millions de dollars**  
  Cela signifie que, en moyenne, la prédiction de votre modèle s'écarte du prix réel d'environ 16,66 millions de dollars.  
  Étant donné que le prix moyen est d'environ 198 millions de dollars, l'erreur moyenne absolue représente environ 8 % du prix moyen.

- **MAPE ≈ 71,8 %**  
  Le MAPE exprime l'erreur en pourcentage par rapport au prix réel. Une valeur aussi élevée indique qu'en moyenne, l'erreur relative par prédiction est importante.  
  Ce résultat peut s'expliquer par la forte hétérogénéité des prix dans votre dataset :  
  - Le prix varie de 145 814,79 dollars à près de 978 millions de dollars.  
  - Pour des avions dont le prix est très bas, une erreur même modeste en valeur absolue peut représenter un pourcentage très élevé.  
  - Le MAPE est donc particulièrement sensible aux cas où le prix réel est faible.

### Que peut-on en conclure ?

- **Performance sur l'ensemble du dataset :**  
  Le R² élevé (≈ 0,98) et le MAE modéré par rapport à la moyenne indiquent que le modèle explique bien la variance des prix et que, sur le plan absolu, l'erreur moyenne n'est pas excessive.

- **Attention aux cas particuliers :**  
  Le MAPE élevé suggère qu'il existe certaines observations (probablement les avions à bas prix) pour lesquelles l'erreur relative est très importante.  
  Il peut être utile de :
  - Visualiser les prédictions par rapport aux valeurs réelles (par exemple, avec un scatter plot) pour identifier les cas problématiques.
  - Analyser la distribution des erreurs selon les segments de prix.
  - Envisager des métriques supplémentaires (comme le MAE, qui est moins sensible aux faibles valeurs) pour avoir une vision complète des performances.


En résumé, vos résultats montrent que le modèle a de bonnes performances globales (R² élevé et MAE acceptable) mais que des écarts relatifs importants apparaissent sur certains cas, notamment pour les avions à prix très bas. Une analyse plus détaillée par segment de prix pourra aider à mieux comprendre et potentiellement améliorer ces aspects.

# Sauvegarder le modele 

In [91]:
!pip install joblib

In [93]:
import joblib

# On sauvegarde le meilleur modèle (pipeline complet) dans un fichier .pkl
joblib.dump(grid_search.best_estimator_, "xgb_airplane_price_model.pkl")

print("Modèle sauvegardé sous 'xgb_airplane_price_model.pkl'")


Modèle sauvegardé sous 'xgb_airplane_price_model.pkl'
